In [1]:
import random
seed_val = 1000
random.seed(seed_val)
import numpy as np
np.random.seed(seed_val)
import tensorflow as tf
tf.set_random_seed(seed_val)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras import regularizers
from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
def conv_model_multi(n_chars, n_consonant, n_vowels, n_units):
    root_word_input = Input(shape=(n_chars, (n_consonant + n_vowels), 1), name="root_word_input")
    
    x = Conv2D(16, (5, 5), padding='same', activation='relu')(root_word_input)
    x = MaxPooling2D(3, 3)(x)

    x = Flatten()(x)

    state_h = Dense(n_units, activation='linear')(x)
    outputs = []
    for i in range(n_chars):
        y = Dense(20, activation='relu')(state_h)
        outputs.append(y)
    con_softs = []
    for i in range(n_chars):
        y = Dense(n_consonant, activation='softmax')(outputs[i])
        con_softs.append(y)
    vow_softs = []
    for i in range(n_chars):
        y = Dense(n_consonant, activation='softmax')(outputs[i])
        vow_softs.append(y)
    model = Model(root_word_input, con)
    
    return main_model, encoder_model, decoder_model

def pred_embeddings_multi(vocab, encoder, char2tup):
    embeddings = np.ndarray((len(vocab), embed_size))
    i = 0
    vec_buffer, con_buffer = [], []
    buffer_size = 10000
    for wi, word in enumerate(vocab):
        word = int2word[word2int[word]]
        convec, vowvec = word2vec_seperated(char2tup, word, n_chars, n_consonant, n_vowel)
#         convec = convec.reshape((-1, n_chars, n_consonant, 1))
#         vowvec = convec.reshape((-1, n_chars, n_vowel, 1))
#         mat = np.concatenate([convec, vowvec], axis=1)
        
        con_buffer.append(convec)
        vec_buffer.append(vowvec)
        if len(vec_buffer) == buffer_size or len(vocab) - wi < buffer_size:
            buffer_np_c = np.stack(con_buffer).reshape((-1, n_chars, n_consonant, 1))
            buffer_np_v = np.stack(vec_buffer).reshape((-1, n_chars, n_vowel, 1))
            result = encoder.predict([buffer_np_c, buffer_np_v])
            embeddings[i:i+len(buffer_np_c)] = result
            i += len(con_buffer)
            vec_buffer, con_buffer = [], []
            if i % (4 *buffer_size) == 0:
                print("Predicting: {0:.2f}%".format((i * 100.0 / len(vocab))))
    print("finished")
    return embeddings


def generateSG(data, word2int,int2word, char2tup, skip_window, batch_size, n_consonant, n_vowels):
    win_size = skip_window  
    i = win_size
    assert batch_size % (win_size) == 0
    targets, target_inputs = {}, {}
    for word in word2int.keys():
        target = word + '|'
        target_input = '&' + target 
        targets[word] = target
        target_inputs[word] = target_input
        
    batch = 0
    n_batchs = len(words) // batch_size
    n_chars = 13
    while True:
        batch_input = []
        batch_decoder_cons_input = []
        batch_decoder_vow_input = []
        batch_output_cons = []
        batch_output_vow = []
        for bi in range(0, batch_size, skip_window * 2 + 1):
            context = data[i - win_size: i + win_size + 1]
            target = [context.pop(win_size)] * (win_size * 2)
#             print(context, target)
            for input_word, output_word in zip(context, target):
#                 print(target_inputs[output_word])
                input_con, input_vow = word2vec_seperated(char2tup, input_word, n_chars, n_consonant, n_vowels)
                target_con, target_vow = word2vec_seperated(char2tup,targets[output_word], n_chars, n_consonant, n_vowels)
                decoder_con, decoder_vow = word2vec_seperated(char2tup,target_inputs[output_word], n_chars, n_consonant, n_vowels)
                input_vec = np.concatenate([input_con, input_vow], axis=1).reshape((n_chars, (n_consonant + n_vowels), 1))
                
                batch_input.append(input_vec)
                batch_decoder_cons_input.append(decoder_con)
                batch_decoder_vow_input.append(decoder_vow)
                batch_output_cons.append(target_con)
                batch_output_vow.append(target_vow)

            i += 1
            if i + win_size + 1 > len(data):
                i = win_size
        batch_input = np.array(batch_input)
        batch_decoder_cons_input = np.array(batch_decoder_cons_input)
        batch_decoder_vow_input = np.array(batch_decoder_vow_input)
        batch_output_cons = np.array(batch_output_cons)
        batch_output_vow = np.array(batch_output_vow)
#         print(batch_input.shape)
        yield [batch_input, batch_decoder_cons_input, batch_decoder_vow_input], [batch_output_cons, batch_output_vow]


In [5]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
int_words = words_to_ints(word2int, words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 300
embed_size = 100
skip_window = 1


n_batches = len(vocab)  // batch_size


In [ ]:
try:
    del multi_train
    del multi_enc
    del multi_dec
    keras.backend.clear_session()
    gc.collect()
except:
    pass
multi_train, multi_enc, multi_dec = conv_model_multi(n_chars, n_consonant, n_vowel, embed_size)
adam = keras.optimizers.Nadam(.001)
multi_train.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
multi_gen = generateSG(words, char2tup, batch_size, n_consonant, n_vowel)
# plot_model(multi_train)
# multi_train.summary()